# <center>Adaptive Exponential Weights Algorithm for <br/> Learning Equilibrium Flows of the Routing Game</center>

In [1]:
# Import necessary libraries
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import random
import networkx as nx   # best working with networkx 2.5 to avoid errors
from scipy import integrate
import uuid   #for generating unique id for files and folders
import csv
import pickle

import os
current_dir= os.getcwd()
import sys
import openmatrix as omx
from itertools import islice

## 1. Preliminary functions for importing edges data (network_net.tntp)

In [2]:
root = os.path.dirname(os.path.abspath('.'))     #Look for the root folder
root = root+'\\TransportationNetworks-master'
root

'C:\\Users\\dongq\\Google Drive\\2021 Works\\Routing game\\Routing-Game-MOR-Submission\\Simulation_python\\TransportationNetworks-master'

In [3]:
def read_edge(network_name):
    netfile = os.path.join(root, network_name,network_name + '_net.tntp')
    net = pd.read_csv(netfile, skiprows=8, sep='\t')

    trimmed= [s.strip().lower() for s in net.columns]
    net.columns = trimmed

    # And drop the silly first andlast columns
    net.drop(['~', ';'], axis=1, inplace=True)
    return net

In [4]:
#Create edgede list from a pandaframe parsed from tntp:
def edge_list(net):
    E = len(net)    #number of edges
    edge_list = []
    for i in range(E): edge_list.append([net['init_node'][i] , net['term_node'][i]])
    return edge_list

In [5]:
### Define the function generating a nx.Graph from Tntp files:
def graph_generation(network_name):
    net = read_edge(network_name)    # Import the whole data_frame of edges data into net
    edges = edge_list(net)           # Generate the list of edges
    G=nx.DiGraph()                     # Create an empty nx.graph
    G.add_edges_from(edges)  # Import edges into the nx.graph
    return G

In [6]:
### Respective to a set of paths, assign on each edge e the indices of paths passing e:
def passing_edges(G,paths):
    N = len(paths)
    # Reset the attribute if already existed
    for e in G.edges(): 
        edge = G[e[0]][e[1]]
        if 'passing_paths' in edge: del edge['passing_paths']
            
    for i in range(N):
        for p in range(len(paths[i])):
            for node in range(len(paths[i][p]) -1):
                e0= paths[i][p][node]
                e1 = paths[i][p][node+1]
                edge = G[e0][e1]    # elif e=(u,v) where v< u; then, its revert edge=(v,u) is in G
                
                if 'passing_paths' not in edge:  edge['passing_paths'] =  [[i,p]]    #If there is not yet any assigned passing_paths, create the attribute
                else:   edge['passing_paths'].append([i,p])

In [7]:
# FUnction to discover k shortest_paths from source to target:
def k_shortest_paths(G, source, target, k, weight=None):
    return list(islice(nx.shortest_simple_paths(G, source, target, weight=weight), k))

## 2. Networks Generation

We will generate g_pickle data for networks by the function ```DAG_generation()``` taking the following arguments:
* ```network_name``` = name of the network from which the data is chosen
* $N \in \mathbb{N}$ = the number of O-D pairs we want to consider
* ```random_pairs``` = whether we randomly choose the O-D pairs or we input by a list of O-D pairs
* ```max_paths``` = maximum number of paths in each DAG
* ```Pairs``` = a data frame indicating the pairs in considerations

The outputs are stored in "./data/network_name".

In [8]:
def DAG_generation(network_name, N, random_pairs = True, Pairs = [], max_paths = 600):  
    G = graph_generation(network_name)    # Create a nx.graph of the network
    print('Create a nx.graph of ', network_name, 'with', nx.info(G))
    V = G.number_of_nodes()
    if random_pairs == False: N = len(Pairs)     # If manually input the Pairs, make sure number of pairs N is correct
    inflow = omx.open_file(root + '\\'+ network_name + '\\demand.omx') #Import the inflows data
    pairs_data = pd.DataFrame(columns = ['source','sink','demand'])
    Paths = [] #Intialize to store the path sets
    
    if random_pairs == True:
        for i in range(N):
            source = random.choice(list(G.nodes))        #Choose randomly a sourcce
            while G.out_degree(source)==0: source = random.choice(list(G.nodes)) # If source has zero out_degree, re-choose it
            sink = random.choice(list(nx.algorithms.dag.descendants(G, source)))   #randomly choose a sink reachable from source
            # if (source,sink) coincides with previous chosen O-D pairs, redo; or if inflow[source][sink]==0, redo:
            while ([source,sink] in Pairs) or (inflow['matrix'][source-1][sink-1]==0):       #Matrix index shifted by 1 
                source = random.choice(list(G.nodes))
                while G.out_degree(source)==0: source = random.choice(list(G.nodes)) # If source has zero out_degree, re-choose it
                sink = random.choice(list(nx.algorithms.dag.descendants(G, source)))   #randomly choose a sink reachable from source
            pairs_data.loc[i] = list([int(source), int(sink),inflow['matrix'][source-1][sink-1]])
            print('Finish creating pairs_data of pair ',i)
           
            Paths_i = k_shortest_paths(G, source, sink, max_paths, weight=None)   #Find max_paths shortest paths from source to sink
#             k = np.random.choice(max_paths+1)
#             Paths_i = k_shortest_paths(G, source, sink, k, weight=None)   #Find max_paths shortest paths from source to sink
            Paths.append(Paths_i)
            print('Finish creating paths data of pair', i)
    
    if random_pairs ==False:
        for i in range(N):
            source = int(Pairs['source'][i])
            sink = int(Pairs['sink'][i])
            if source > V or sink > V: print('Wrong input of pairs')
            else:
                pairs_data.loc[i] = list([int(source), int(sink),Pairs['demand'][i]])
                print('Finish creating pairs_data of pair ',i)
    
                Paths_i = k_shortest_paths(G, source, sink, max_paths, weight=None)   #Find max_paths shortest paths from source to sink
                Paths.append(Paths_i)
                print('Finish creating paths data of pair', i)
    
    #Store the pairs_data   
    directory = current_dir+"/data/"+network_name+'/'
    os.makedirs(os.path.dirname(directory), exist_ok=True)
    pairs_data.to_csv(directory+'pairs,N='+str(N)+'.csv')

    #Store the path data   
    with open(directory+'paths,N='+str(N)+'.data', 'wb') as filehandle:
        pickle.dump(Paths, filehandle)  # STORE the data as binary data stream into data/network_name/
        
     
    print('Pre-processing the graph data')
    # Pre-process the graph and store the graph   
    # Assign each edge e in G with indices of the paths (relevant to the choice of pairs) passing through it
    passing_edges(G,Paths)
    nx.write_gpickle(G,directory +"/graph,N="+ str(N)+".gpickle")   

### 1.1. Test: DAG generations with SiouxFalls

In [16]:
N = 10
DAG_generation('Austin', N, random_pairs = True, Pairs=[], max_paths = 10)

Create a nx.graph of  Austin with Name: 
Type: DiGraph
Number of nodes: 7388
Number of edges: 18956
Average in degree:   2.5658
Average out degree:   2.5658
Finish creating pairs_data of pair  0
Finish creating paths data of pair 0
Finish creating pairs_data of pair  1
Finish creating paths data of pair 1
Finish creating pairs_data of pair  2
Finish creating paths data of pair 2
Finish creating pairs_data of pair  3
Finish creating paths data of pair 3
Finish creating pairs_data of pair  4
Finish creating paths data of pair 4
Finish creating pairs_data of pair  5
Finish creating paths data of pair 5
Finish creating pairs_data of pair  6
Finish creating paths data of pair 6
Finish creating pairs_data of pair  7
Finish creating paths data of pair 7
Finish creating pairs_data of pair  8
Finish creating paths data of pair 8
Finish creating pairs_data of pair  9
Finish creating paths data of pair 9
Pre-processing the graph data
